In [ ]:
'''
 * Copyright 2023 LLM-Info (?????????????????????)
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *    http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 '''

# Generating Consistent and High Quality Infoboxes with LLMs

In [16]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
from scipy import spatial  # for calculating vector similarities for search


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

from config import *
openai.api_key = OPENAI_API_KEY

# Vector Database

As input, we get raw Wikipedia text. As output, we will get the most similar infobox template.

NOTE: do we want to structure the template as a bunch of fields or in proper format?

In [ ]:
# search function - replace with vdb function
def fake_vdb(
    query: str,
    templates: list,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 1
):
    '''"""Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]'''
    return templates[0]


In [1]:
import re
import os
import chromadb
import numpy as np
from tqdm import tqdm
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document

In [2]:
client = chromadb.PersistentClient(path="./QueryPipeline/")

collection = client.get_collection(name="infoboxes")

langchain_chroma = Chroma(
    client=client,
    collection_name="infoboxes",
    embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"),
)
print("Success!")

c:\Users\ishik\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Success!


In [3]:
def retrieve_template_from_vdb(query):
    template = langchain_chroma.similarity_search_with_score(query)
    return template

In [4]:
def retrieve_template_from_object(template):
    template = template[0].metadata['source'] # extract the infobox template
    template = template[template.index("Infobox_"):] # remove random seo stuff

    # construct template consistent with wikipedia's format
    template = template.split(' ')
    template_string = "{{" + template[0] + "\n"
    for field in template[1:]:
        template_string += f'| {field} = \n'
    template_string += "}}"
    return template_string

In [13]:
# Example
query = """
algorithm
"""
retrieve_template_from_vdb(query)

[(Document(page_content='Infobox_algorithm', metadata={'source': 'Infobox algorithm Infobox algorithm Infobox algorithm Infobox algorithm Infobox algorithm  Infobox_algorithm name class image caption data time best-time average-time space\n'}),
  1.0512022972106934),
 (Document(page_content='Infobox_artificial_intelligence', metadata={'source': 'Infobox artificial intelligence Infobox artificial intelligence Infobox artificial intelligence Infobox artificial intelligence Infobox artificial intelligence  Infobox_artificial_intelligence name logo image caption developer user country introduced type purpose language derived_from replaced_by website\n'}),
  1.444591999053955),
 (Document(page_content='Infobox_mathematical_function', metadata={'source': 'Infobox mathematical function Infobox mathematical function Infobox mathematical function Infobox mathematical function Infobox mathematical function  Infobox_mathematical_function name image= imagesize= imagealt= parity= domain= codomain= 

# LLM Querying

As input, we provide the raw Wikipedia text and the infobox template. As output, we receive the generated infobox.

In [17]:
def formulate_query(
    article: str,
    infobox_template: str
) -> str:
    introduction = "Your task is to fill out a Wikipedia infobox. Below, you are given some context text and the infobox template.\n"
    question = f"\n\n Context text: {article}\n\n Infobox Template: {infobox_template} \n\n"
    end = 'Fill out the Wikipedia infobox. Feel free to add any fields that you think would be important to know. Remember, keep the infobox concise, accurate, and of good quality.'
    return introduction + question + end

def api_call(message, model: str = GPT_MODEL):
    messages = [
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

def ask(
    article: str,
    infobox_template: str,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = formulate_query(article, infobox_template)
    if print_message:
        print(message)
    
    reply = 'lol' #api_call(message, model)
    return reply

# Pipeline

In [26]:
fake = ['mythical creature']

In [27]:
articles = ["""
"""]

In [28]:
for template, article in zip(fake, articles):
    # classification
    template = retrieve_template_from_vdb(template)
    for t in template:
        print(t)

    num = int(input('which one?'))
    template = retrieve_template_from_object(template[num])

    # generation
    infobox = ask(article, template, print_message=True)
    

(Document(page_content='Infobox_mythical_creature', metadata={'source': 'Infobox mythical creature Infobox mythical creature Infobox mythical creature Infobox mythical creature Infobox mythical creature  Infobox_mythical_creature name image image_size image_upright caption Grouping Sub_Grouping Similar_entities Family Folklore First_Attested AKA Country Region Habitat Details\n'}), 0.5487587451934814)
(Document(page_content='Infobox_Primeval_creature', metadata={'source': 'Infobox Primeval creature Infobox Primeval creature Infobox Primeval creature Infobox Primeval creature Infobox Primeval creature  Infobox_Primeval_creature name image species period appeared first_primeval last_primeval first_new_world last_new_world number humans_killed returned\n'}), 1.1872961521148682)
(Document(page_content='Infobox_monster_truck', metadata={'source': 'Infobox monster truck Infobox monster truck Infobox monster truck Infobox monster truck Infobox monster truck  Infobox_monster_truck name image i

Your task is to fill out a Wikipedia infobox. Below, you are given some context text and the infobox template.


 Context text: 


 Infobox Template: {{Infobox_mythical_creature
| name = 
| image = 
| image_size = 
| image_upright = 
| caption = 
| Grouping = 
| Sub_Grouping = 
| Similar_entities = 
| Family = 
| Folklore = 
| First_Attested = 
| AKA = 
| Country = 
| Region = 
| Habitat = 
| Details
 = 
}} 

Fill out the Wikipedia infobox. Feel free to add any fields that you think would be important to know. Remember, keep the infobox concise, accurate, and of good quality.
